## Preprocesado de datos de centros sanitarios

Este notebook se dedica a la **carga y preprocesado de los datos de centros sanitarios** del Sistema Valenciano de Salud en la Comunitat Valenciana.

El objetivo es limpiar, depurar y estructurar la información de **localización geográfica y tipología de los centros sanitarios** (hospitales, centros de salud y centros de especialidades), preparando un dataset homogéneo y coherente para su posterior integración con los datos demográficos y territoriales a nivel municipal.

Los datos proceden de distintos conjuntos publicados por la Generalitat Valenciana y se unifican en un único dataset, manteniendo únicamente las variables relevantes para el análisis territorial.

- Datos abiertos de la Generalitat Valenciana – Sistema Valenciano de Salud (centros sanitarios)

In [1]:
import pandas as pd

### Carga de datos y exploración inicial

#### Carga de los datasets de centros sanitarios

Se cargan de forma independiente los tres conjuntos de datos del Sistema Valenciano de Salud 
(hospitales, centros de salud y centros de especialidades), que posteriormente se integrarán 
en un único dataset.

In [2]:
# Crear 3 dataframes de centros sanitarios
hospitales_raw = pd.read_csv('../data/raw/GVA-hospitales.csv', sep=',')
centros_salud_raw = pd.read_csv('../data/raw/GVA-centros-de-salud.csv', sep=',')
centros_especialidades_raw = pd.read_csv('../data/raw/GVA-centros-de-especialidades.csv', sep=',')

#### Comprobaciones iniciales

Antes de integrar los datasets, se revisan sus dimensiones y la coherencia de sus estructuras, 
verificando que todos comparten el mismo esquema de variables.

In [3]:
# Dimensión de datasets
print(hospitales_raw.shape)
print(centros_salud_raw.shape)
print(centros_especialidades_raw.shape)

(35, 21)
(248, 21)
(26, 21)


In [4]:
# Comprobar que las columnas son las mismas en los 3 datasets
print(all(hospitales_raw.columns == centros_salud_raw.columns) and all(hospitales_raw.columns == centros_especialidades_raw.columns))
print(hospitales_raw.columns)

True
Index(['WKT', 'id', 'id_interno', 'cen_cod', 'cen_desclar', 'cod_ine_mun',
       'municipio', 'provincia', 'cen_nombcall', 'cen_numcall', 'cen_codpos',
       'tce_codi', 'codigo_departamento', 'nombre_departamento', 'codigo_zona',
       'nombre_zona', 'cen_feci', 'x_coord', 'y_coord', 'tipo', 'tipo_val'],
      dtype='object')


#### Integración y exploración inicial

Una vez verificada la compatibilidad de los tres datasets, se concatenan en un único dataframe 
que recoge la totalidad de centros sanitarios del Sistema Valenciano de Salud.  
Sobre este conjunto integrado se realiza una primera exploración descriptiva.

In [5]:
# Concatenar los 3 dataframes en uno solo
centros_raw = pd.concat([hospitales_raw, centros_salud_raw, centros_especialidades_raw], ignore_index=True)

In [6]:
# Dimension del dataframe combinado
centros_raw.shape

(309, 21)

In [7]:
# Valores nulos por columna
centros_raw.isna().sum()

WKT                      0
id                       0
id_interno               0
cen_cod                  0
cen_desclar              0
cod_ine_mun              0
municipio                0
provincia                0
cen_nombcall             0
cen_numcall              8
cen_codpos               0
tce_codi                 0
codigo_departamento      0
nombre_departamento      6
codigo_zona              0
nombre_zona             61
cen_feci               309
x_coord                  0
y_coord                  0
tipo                     0
tipo_val                 0
dtype: int64

In [8]:
# Número de centros por tipo
centros_raw['tipo'].value_counts()

tipo
Centros de salud             248
Hospitales                    35
Centros de especialidades     26
Name: count, dtype: int64

In [9]:
# Número de municipios distintos
centros_raw['cod_ine_mun'].nunique()

168

In [10]:
# Descripción estadística del dataset
centros_raw.describe(include='all')

,WKT,id,id_interno,cen_cod,cen_desclar,cod_ine_mun,municipio,provincia,cen_nombcall,cen_numcall,...,tce_codi,codigo_departamento,nombre_departamento,codigo_zona,nombre_zona,cen_feci,x_coord,y_coord,tipo,tipo_val
count,309,309.000000,309.000000,309.000000,309,309.000000,309,309,309,301,...,309.000000,309.000000,303,309.000000,248,0.0,309.000000,3.090000e+02,309,309
unique,308,NaN,NaN,NaN,309,NaN,168,3,294,77,...,NaN,NaN,24,NaN,226,NaN,NaN,NaN,3,3
top,POINT (715662.76062 4253467.08584),NaN,NaN,NaN,HOSPITAL PSIQUIATRICO PROVINCIAL,NaN,València,València/Valencia,AUSIAS MARCH,S/N,...,NaN,NaN,CASTELLON,NaN,SANT AGUSTI (CASTELLO),NaN,NaN,NaN,Centros de salud,Centres de salut
freq,2,NaN,NaN,NaN,1,NaN,42,157,3,116,...,NaN,NaN,27,NaN,3,NaN,NaN,NaN,248,248
mean,NaN,155.000000,186700.268608,1101.731392,NaN,26480.155340,NaN,NaN,NaN,NaN,...,3.650485,10.951456,NaN,1100.864078,NaN,NaN,723065.939569,4.469957e+06,NaN,NaN
std,NaN,89.344838,76988.940700,5194.319517,NaN,20264.201041,NaN,NaN,NaN,NaN,...,5.357436,6.806288,NaN,679.854153,NaN,NaN,23752.249734,2.281538e+06,NaN,NaN
min,NaN,1.000000,26157.000000,2.000000,NaN,3002.000000,NaN,NaN,NaN,NaN,...,1.000000,0.000000,NaN,0.000000,NaN,NaN,646247.848321,4.198385e+06,NaN,NaN
25%,NaN,78.000000,224699.000000,233.000000,NaN,3102.000000,NaN,NaN,NaN,NaN,...,1.000000,5.000000,NaN,509.000000,NaN,NaN,709674.948963,4.278917e+06,NaN,NaN
50%,NaN,155.000000,224776.000000,457.000000,NaN,46007.000000,NaN,NaN,NaN,NaN,...,1.000000,11.000000,NaN,1100.000000,NaN,NaN,722937.129818,4.365239e+06,NaN,NaN
75%,NaN,232.000000,224853.000000,959.000000,NaN,46184.000000,NaN,NaN,NaN,NaN,...,1.000000,17.000000,NaN,1700.000000,NaN,NaN,734766.742090,4.382127e+06,NaN,NaN


Se observa que el dataset final incluye centros sanitarios distribuidos en un número amplio 
de municipios de la Comunitat Valenciana, lo que permite un análisis territorial consistente.

In [11]:
# Vista previa del dataframe
centros_raw.sample(5)

,WKT,id,id_interno,cen_cod,cen_desclar,cod_ine_mun,municipio,provincia,cen_nombcall,cen_numcall,...,tce_codi,codigo_departamento,nombre_departamento,codigo_zona,nombre_zona,cen_feci,x_coord,y_coord,tipo,tipo_val
26,POINT (719179.32232 4373738.73047),275,35952,2837,HOSPITAL DE MANISES,46159,Manises,València/Valencia,AVDA. GENERALITAT VALENCIANA,50,...,14,23,MANISES,2300,NaN,NaN,719179.322320,4.373739e+06,Hospitales,Hospitals
286,POINT (743529.0022 4316778.21127),287,26162,955,C.E. DE GANDIA,46131,Gandia,València/Valencia,AVINGUDA DE LA MEDICINA,6,...,15,12,GANDIA,1200,NaN,NaN,743529.002196,4.316778e+06,Centros de especialidades,Centres d'especialitats
244,POINT (724471.16229 4369794.62476),210,224892,707,CS V PLAZA SEGOVIA,46250,València,València/Valencia,PZA SEGOVIA,S/N,...,1,10,VALENCIA - DR. PESET,1012,PLAZA SEGOVIA (VALENCIA),NaN,724471.162290,4.369795e+06,Centros de salud,Centres de salut
143,POINT (723335.54327 4365238.96895),109,224791,299,CS CATARROJA,46094,Catarroja,València/Valencia,AVDA. RAMBLETA,63,...,1,7,VALENCIA - LA FE,716,CATARROJA,NaN,723335.543267,4.365239e+06,Centros de salud,Centres de salut
88,POINT (646247.84832 4436297.60297),54,224736,174,CS ADEMUZ,46001,Ademuz,València/Valencia,AVDA. VALENCIA,68,...,1,6,VALENCIA ARNAU LLIRIA,601,ADEMUZ,NaN,646247.848321,4.436298e+06,Centros de salud,Centres de salut


### Análisis exploratorio y preparación de variables

#### Selección de variables

Tras el análisis exploratorio del dataset completo de centros sanitarios, se decide conservar únicamente las siguientes variables:

- `WKT`: geometría puntual del centro sanitario, utilizada para análisis espaciales, asignación territorial a municipios y representación cartográfica.
- `cen_cod`: código identificador del centro sanitario dentro del Sistema Valenciano de Salud, que permite distinguir de forma unívoca cada centro.
- `cen_desclar`: denominación declarada del centro sanitario. Permite identificar y describir el equipamiento, así como detectar posibles duplicidades o variaciones administrativas en la denominación de un mismo centro.
- `cod_ine_mun`: código oficial del municipio según el INE, utilizado como identificador geográfico principal para la integración con los datasets demográficos y territoriales.
- `tipo`: tipología del centro sanitario (hospital, centro de salud o centro de especialidades), utilizada para clasificar y analizar la oferta sanitaria por tipo de recurso.

Las variables de carácter administrativo interno, direcciones postales detalladas, coordenadas duplicadas o identificadores secundarios que no aportan valor añadido al análisis territorial se excluyen del dataset final. Esta selección permite simplificar la estructura del dataset y centrar el análisis en las dimensiones geográfica y funcional relevantes para el estudio.

In [12]:
centros_clean = centros_raw[['cen_cod', 'cen_desclar', 'cod_ine_mun', 'tipo', 'WKT']]
centros_clean.describe(include='all')

,cen_cod,cen_desclar,cod_ine_mun,tipo,WKT
count,309.000000,309,309.000000,309,309
unique,NaN,309,NaN,3,308
top,NaN,HOSPITAL PSIQUIATRICO PROVINCIAL,NaN,Centros de salud,POINT (715662.76062 4253467.08584)
freq,NaN,1,NaN,248,2
mean,1101.731392,NaN,26480.155340,NaN,NaN
std,5194.319517,NaN,20264.201041,NaN,NaN
min,2.000000,NaN,3002.000000,NaN,NaN
25%,233.000000,NaN,3102.000000,NaN,NaN
50%,457.000000,NaN,46007.000000,NaN,NaN
75%,959.000000,NaN,46184.000000,NaN,NaN


#### Variable `WKT`

La variable `WKT` representa la geometría puntual de cada centro sanitario en formato *Well-Known Text*. Se utiliza para el análisis espacial, la representación cartográfica y la asignación territorial de los centros a los municipios.

Según la descripción del dataset, existen **309 registros** y **308 valores únicos** en esta variable, lo que indica la presencia de **dos filas que comparten exactamente la misma geometría**. El análisis detallado de estos registros muestra que corresponden al mismo centro sanitario, ubicado en la misma dirección, municipio y con la misma tipología, diferenciándose únicamente en la denominación administrativa.

Con el fin de evitar la duplicación de equipamientos en los análisis espaciales y territoriales posteriores, se elimina uno de los registros duplicados, conservando un único registro por geometría puntual.

In [13]:
# Ver filas con wkt duplicados
centros_clean[centros_clean.duplicated(subset=['WKT'], keep=False)]

,cen_cod,cen_desclar,cod_ine_mun,tipo,WKT
284,4138,CENTRO DE ESPECIALIDADES SANT VICENT DEL RASPE...,3122,Centros de especialidades,POINT (715662.76062 4253467.08584)
305,2753,C.E. DE SAN VTE. RASPEIG,3122,Centros de especialidades,POINT (715662.76062 4253467.08584)


In [14]:
# Eliminar fila duplicada
centros_clean = centros_clean[centros_clean['cen_cod'] != 2753].reset_index(drop=True)
centros_clean.describe(include='all')

,cen_cod,cen_desclar,cod_ine_mun,tipo,WKT
count,308.000000,308,308.000000,308,308
unique,NaN,308,NaN,3,308
top,NaN,HOSPITAL PSIQUIATRICO PROVINCIAL,NaN,Centros de salud,POINT (719542.884 4383884.783)
freq,NaN,1,NaN,248,1
mean,1096.370130,NaN,26555.993506,NaN,NaN
std,5201.916044,NaN,20253.208298,NaN,NaN
min,2.000000,NaN,3002.000000,NaN,NaN
25%,232.750000,NaN,3101.750000,NaN,NaN
50%,456.500000,NaN,46009.000000,NaN,NaN
75%,958.250000,NaN,46184.000000,NaN,NaN


#### Variable `cen_cod`

La variable `cen_cod` es un **identificador numérico del centro sanitario** dentro del sistema de información de la Generalitat Valenciana.

- Es un código interno utilizado para identificar de forma única a cada centro sanitario.
- No presenta valores nulos en el dataset.
- Su función principal es servir como **identificador del centro**.

En el presente proyecto, `cen_cod` se conserva únicamente como **identificador técnico**, pero no se utiliza como variable analítica.

In [15]:
centros_clean['cen_cod'].isna().sum()

np.int64(0)

In [16]:
centros_clean['cen_cod'].nunique(), len(centros_clean)

(308, 308)

#### Variable `cen_desclar`

La variable `cen_desclar` recoge la denominación declarada del centro sanitario, es decir, el nombre oficial con el que el centro figura en el registro del Sistema Valenciano de Salud.

Aunque esta variable no se utiliza directamente en los análisis cuantitativos ni espaciales agregados, se conserva en el dataset porque permite identificar nominalmente cada centro sanitario y puede ser relevante para futuras ampliaciones del proyecto o análisis más detallados a nivel de centro.

No se realiza ninguna transformación sobre esta variable en la fase de preprocesado, manteniéndose tal y como aparece en el dataset original.

In [17]:
centros_clean['cen_desclar'].isna().sum()

np.int64(0)

In [18]:
centros_clean['cen_desclar'].nunique(), len(centros_clean)

(308, 308)

#### Variable `cod_ine_mun`

La variable `cod_ine_mun` corresponde al **código oficial del municipio según el INE** y se utiliza como **identificador clave** para la integración del dataset de centros sanitarios con los datos de población y delimitaciones municipales.

El análisis de los valores únicos muestra que los registros de centros sanitarios abarcan **168 municipios distintos** de la Comunitat Valenciana. Esta cobertura territorial se considera **suficiente para los objetivos del estudio**.

La ausencia de centros en algunos municipios puede explicarse por:
- la **centralización de los servicios sanitarios** en municipios de mayor tamaño,
- la **dependencia funcional** de municipios pequeños respecto a centros ubicados en localidades cercanas,
- el hecho de que el dataset recoge únicamente centros del **Sistema Valenciano de Salud**, excluyendo consultorios temporales o centros privados.

In [19]:
centros_clean['cod_ine_mun'].nunique()

168

#### Variable `tipo`

La variable `tipo` identifica la **tipología del centro sanitario** y permite clasificar los registros según el nivel y tipo de atención ofrecida.

En el dataset se distinguen **tres categorías principales**:
- **Centros de salud**
- **Centros de especialidades**
- **Hospitales**

Esta variable es clave para el análisis, ya que permite:
- diferenciar la **oferta sanitaria básica y especializada**,
- calcular indicadores desagregados por tipo de centro (por ejemplo, centros de salud por municipio),
- analizar la **distribución territorial** de los distintos niveles asistenciales.

Se mantiene sin transformación adicional, al tratarse de una variable categórica clara y consistente.

In [20]:
centros_clean['tipo'].unique()

array(['Hospitales', 'Centros de salud', 'Centros de especialidades'],
      dtype=object)

In [21]:
# Número de centros por tipo
centros_clean['tipo'].value_counts()

tipo
Centros de salud             248
Hospitales                    35
Centros de especialidades     25
Name: count, dtype: int64

In [22]:
# Convertir a variable categórica
centros_clean['tipo'] = centros_clean['tipo'].astype('category')
centros_clean.describe(include='all')

,cen_cod,cen_desclar,cod_ine_mun,tipo,WKT
count,308.000000,308,308.000000,308,308
unique,NaN,308,NaN,3,308
top,NaN,HOSPITAL PSIQUIATRICO PROVINCIAL,NaN,Centros de salud,POINT (719542.884 4383884.783)
freq,NaN,1,NaN,248,1
mean,1096.370130,NaN,26555.993506,NaN,NaN
std,5201.916044,NaN,20253.208298,NaN,NaN
min,2.000000,NaN,3002.000000,NaN,NaN
25%,232.750000,NaN,3101.750000,NaN,NaN
50%,456.500000,NaN,46009.000000,NaN,NaN
75%,958.250000,NaN,46184.000000,NaN,NaN


### Normalización de nombres de columnas

Con el objetivo de facilitar la integración con otros datasets, mejorar la legibilidad del código y evitar problemas derivados del uso de espacios, acentos o caracteres especiales, se procede a normalizar los nombres de las columnas del dataset de población.

In [26]:
centros_clean = centros_clean.rename(columns={
    'cod_ine_mun': 'cod_municipio',
    'WKT': 'wkt_centro'
})

centros_clean.columns


Index(['cen_cod', 'cen_desclar', 'cod_municipio', 'tipo', 'wkt_centro'], dtype='object')

### Exportación del dataset de municipios

Una vez finalizado el proceso de limpieza, transformación y filtrado del conjunto de datos de centros sanitarios, se procede a guardar el dataset resultante en la carpeta `data/processed`. Este conjunto de datos limpio y estructurado será utilizado posteriormente en la fase de integración con las demás fuentes de información del proyecto.

El archivo se almacena en formato CSV para facilitar su reutilización en futuros análisis o proyectos.

In [27]:
centros_clean.sample(5)

,cen_cod,cen_desclar,cod_municipio,tipo,wkt_centro
138,241,C.S. V AZUCENA,46250,Centros de salud,POINT (724402.97949 4374794.18234)
200,544,CENTRO DE SALUD DE ALMORADI,3015,Centros de salud,POINT (692964.71357 4219727.17236)
64,71,CS MONTANEJOS,12079,Centros de salud,POINT (711010.92078 4438135.16117)
196,396,CENTRO DE SALUD DE PEDREGUER,3101,Centros de salud,POINT (763331.82806 4298234.2759)
68,30,CS ATZENETA MAESTRAT,12001,Centros de salud,POINT (740911.4073 4455448.09874)


In [28]:
centros_clean.to_csv('../data/processed/GVA-centros-sanitarios-clean.csv', index=False)